In [1]:
#import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import ast 

In [2]:
# Reading the TSV file into a DataFrame
data = pd.read_csv("chipotle.tsv", sep='\t')

In [3]:
#creating a dataframe for the data
df=pd.DataFrame(data)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [5]:
df.shape

(4622, 5)

In [6]:
df.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [7]:
#Changing the column names (Representing as A Title)
new_column_names = {
    'order_id':'Order_Id',
    'quantity':'Quantity',
    'choice_description': 'Choice_Description',
    'item_name': 'Item_Name',
    'item_price':'Item_Price'
}

df.rename(columns=new_column_names, inplace=True)


In [8]:
df

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


# 1. Missing Values:
Question: Check for missing values in each column (Order ID, Quantity, Item Name, Choice Description, Item Price). How should missing values be handled?


In [9]:
missing_values = df.isnull().sum()
print("Missing Values in Each Column:")
print(missing_values)

Missing Values in Each Column:
Order_Id                 0
Quantity                 0
Item_Name                0
Choice_Description    1246
Item_Price               0
dtype: int64


The column 'Choice_Description' contains 1246 missing values, while other columns such as 'Order_Id', 'Quantity', 'Item_Name', and 'Item_Price' have no missing values, as indicated by the count of null values using the isnull().sum() 

In [10]:
# Fill missing values in 'choice_description' with "No description"
df['Choice_Description'].fillna("None", inplace=True)
df.head(10)

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,None,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [11]:
#verifying If there is anyy null values after filling the Nan
missing_values = df.isnull().sum()
missing_values

Order_Id              0
Quantity              0
Item_Name             0
Choice_Description    0
Item_Price            0
dtype: int64

# 2. Data Types:
Question: Verify the data types of each column. Do they align with their expected types, and should any adjustments be made?


In [12]:
df.dtypes

Order_Id               int64
Quantity               int64
Item_Name             object
Choice_Description    object
Item_Price            object
dtype: object

In [13]:
#Converting the item_price to float data type
# Remove the dollar sign ($) and convert 'item_price' to float
df['Item_Price'] = df['Item_Price'].str.replace('$', '').astype(float)


C:\Users\91638\AppData\Local\Temp\ipykernel_5728\550195653.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Item_Price'] = df['Item_Price'].str.replace('$', '').astype(float)


In [14]:
#verifying if the data type of item_price has been converted to float
df.dtypes

Order_Id                int64
Quantity                int64
Item_Name              object
Choice_Description     object
Item_Price            float64
dtype: object

# 3. Duplicated Entries:
Question: Identify and handle duplicated entries in the dataset. How might duplicates impact analysis, and what is the appropriate action?


In [15]:
#No of Rows before removing the duplicates
df.shape

(4622, 5)

In [16]:
# Finding duplicates based on specific columns (e.g., Order_Id and Item_Name ,Choice_Description)
df[df.duplicated(subset=['Order_Id', 'Item_Name' ,'Choice_Description'], keep=False)]

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price
234,103,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Rice, Black Beans...",11.75
238,103,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Rice, Black Beans...",11.75
246,108,1,Canned Soda,[Mountain Dew],1.09
248,108,1,Canned Soda,[Mountain Dew],1.09
296,129,1,Steak Burrito,"[Tomatillo Green Chili Salsa, [Rice, Cheese, G...",11.75
...,...,...,...,...,...
4544,1806,1,Bottled Water,None,1.50
4568,1817,1,Bottled Water,None,1.50
4570,1817,1,Bottled Water,None,1.50
4602,1827,1,Barbacoa Burrito,[Tomatillo Green Chili Salsa],9.25


In [17]:
#Dropping the duplicates
df.drop_duplicates(subset=['Order_Id', 'Item_Name' , 'Choice_Description'], keep=False, inplace=True)
df

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price
0,1,1,Chips and Fresh Tomato Salsa,None,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


In [18]:
#No of rows after removing the duplicates
df.shape          #4622(original dataset rows)-117(duplicates)= 4505rows(after removing the duplicates)

(4505, 5)

# 4. Quantity and Item Price:
Question: Examine the Quantity and Item Price columns. Are there any inconsistencies or anomalies that need correction?


In [19]:
df.head(10)

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price
0,1,1,Chips and Fresh Tomato Salsa,None,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98
6,3,1,Side of Chips,None,1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25


In [20]:
# Checking for negative or zero values in Quantity column
negative_quantity = df[df['Quantity'] <= 0]
print("Rows with negative or zero Quantity values:")
print(negative_quantity)

Rows with negative or zero Quantity values:
Empty DataFrame
Columns: [Order_Id, Quantity, Item_Name, Choice_Description, Item_Price]
Index: []


In [21]:
# Checking for negative or zero values in Item Price column
negative_price = df[df['Item_Price'] <= 0]
print("Rows with negative or zero Item Price values:")
print(negative_price)

Rows with negative or zero Item Price values:
Empty DataFrame
Columns: [Order_Id, Quantity, Item_Name, Choice_Description, Item_Price]
Index: []


In [22]:
# Summary statistics for Quantity column
(df['Quantity'].describe())

count    4505.000000
mean        1.076138
std         0.405580
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        15.000000
Name: Quantity, dtype: float64

In [23]:
# Summary statistics for Item Price column
(df['Item_Price'].describe())


count    4505.000000
mean        7.514337
std         4.243410
min         1.090000
25%         3.990000
50%         8.750000
75%         9.390000
max        44.250000
Name: Item_Price, dtype: float64

Quantity: The data seems to predominantly consist of orders with a quantity of 1, as indicated by the quartiles and mean.

Item Price: Prices vary notably, ranging from 1.09 to  44.25, with a mean of  7.51.

Quantity Column: The statistics show no negative or zero values, and the range (1 to 15) seems reasonable.

Item Price Column: The prices also seem within a reasonable range without any negative values. However, there's variability and some higher-priced items, but nothing indicating an obvious issue.

Overall, based on these statistics, the data seems generally consistent within expected ranges for both Quantity and Item Price columns.

# 5. Choice Description:
Question: Analyze the Choice Description column. How should choices be handled, especially when there are multiple descriptions for a single item?


In [24]:
df['Choice_Description'].head(20)

0                                                  None
1                                          [Clementine]
2                                               [Apple]
3                                                  None
4     [Tomatillo-Red Chili Salsa (Hot), [Black Beans...
5     [Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...
6                                                  None
7     [Tomatillo Red Chili Salsa, [Fajita Vegetables...
8     [Tomatillo Green Chili Salsa, [Pinto Beans, Ch...
9     [Fresh Tomato Salsa, [Rice, Black Beans, Pinto...
10                                                 None
11    [Roasted Chili Corn Salsa, [Fajita Vegetables,...
12    [Roasted Chili Corn Salsa, [Rice, Black Beans,...
13    [Fresh Tomato Salsa, [Fajita Vegetables, Rice,...
14                                                 None
15                                                 None
16    [Tomatillo-Green Chili Salsa (Medium), [Pinto ...
17    [Fresh Tomato Salsa (Mild), [Black Beans, 

In [25]:
#Splitting the Choice_Description column
split_columns = df['Choice_Description'].str.split(",", expand=True)


split_columns.columns = ['Choice_Description_1', 'Choice_Description_2', 'Choice_Description_3',
                         'Choice_Description_4', 'Choice_Description_5', 'Choice_Description_6',
                         'Choice_Description_7', 'Choice_Description_8', 'Choice_Description_9',
                         'Choice_Description_10']


df = pd.concat([df, split_columns], axis=1)

In [26]:
df

,Order_Id,Quantity,Item_Name,Choice_Description,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,None,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,[Clementine],3.39,[Clementine],None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,[Apple],3.39,[Apple],None,None,None,None,None,None,None,None,None
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,[Tomatillo-Red Chili Salsa (Hot),[Black Beans,Rice,Cheese,Sour Cream]],None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75,[Fresh Tomato Salsa,[Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole]],None,None,None
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75,[Fresh Tomato Salsa,[Rice,Sour Cream,Cheese,Lettuce,Guacamole]],None,None,None,None
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,[Fresh Tomato Salsa,[Fajita Vegetables,Pinto Beans,Guacamole,Lettuce]],None,None,None,None,None
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,[Fresh Tomato Salsa,[Fajita Vegetables,Lettuce]],None,None,None,None,None,None,None


In [27]:
#dropping the Choice_Desciption column
df.drop('Choice_Description', axis=1, inplace=True)

In [28]:
df

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,[Clementine],None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,[Apple],None,None,None,None,None,None,None,None,None
3,1,1,Chips and Tomatillo-Green Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,[Tomatillo-Red Chili Salsa (Hot),[Black Beans,Rice,Cheese,Sour Cream]],None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,11.75,[Fresh Tomato Salsa,[Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole]],None,None,None
4618,1833,1,Steak Burrito,11.75,[Fresh Tomato Salsa,[Rice,Sour Cream,Cheese,Lettuce,Guacamole]],None,None,None,None
4619,1834,1,Chicken Salad Bowl,11.25,[Fresh Tomato Salsa,[Fajita Vegetables,Pinto Beans,Guacamole,Lettuce]],None,None,None,None,None
4620,1834,1,Chicken Salad Bowl,8.75,[Fresh Tomato Salsa,[Fajita Vegetables,Lettuce]],None,None,None,None,None,None,None


# 6. Handling Special Characters:
Question: Check for special characters in text-based columns (e.g., Item Name, Choice Description). How can these be addressed for consistency?



In [29]:
columns_to_check = ['Item_Name', 'Choice_Description_1','Choice_Description_2','Choice_Description_3','Choice_Description_4','Choice_Description_5','Choice_Description_6','Choice_Description_7','Choice_Description_8','Choice_Description_9','Choice_Description_10']

In [30]:
# Regular expression to identify special characters
special_chars = re.compile(r'[^a-zA-Z0-9\s]')

In [31]:
# Apply handling to specified columns using applymap and lambda
df[columns_to_check] = df[columns_to_check].applymap(lambda x: special_chars.sub('', x) if isinstance(x, str) else x)

In [32]:
df.head(10)

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,Apple,None,None,None,None,None,None,None,None,None
3,1,1,Chips and TomatilloGreen Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None
5,3,1,Chicken Bowl,10.98,Fresh Tomato Salsa Mild,Rice,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None,None
6,3,1,Side of Chips,1.69,None,None,None,None,None,None,None,None,None,None
7,4,1,Steak Burrito,11.75,Tomatillo Red Chili Salsa,Fajita Vegetables,Black Beans,Pinto Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None
8,4,1,Steak Soft Tacos,9.25,Tomatillo Green Chili Salsa,Pinto Beans,Cheese,Sour Cream,Lettuce,None,None,None,None,None
9,5,1,Steak Burrito,9.25,Fresh Tomato Salsa,Rice,Black Beans,Pinto Beans,Cheese,Sour Cream,Lettuce,None,None,None


In [33]:
# Checking for special characters in specified columns using str.contains()
special_chars_check = df[columns_to_check].apply(lambda col: col.str.contains(special_chars))
special_chars_check

,Item_Name,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,False,False,None,None,None,None,None,None,None,None,None
1,False,False,None,None,None,None,None,None,None,None,None
2,False,False,None,None,None,None,None,None,None,None,None
3,False,False,None,None,None,None,None,None,None,None,None
4,False,False,False,False,False,False,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
4617,False,False,False,False,False,False,False,False,None,None,None
4618,False,False,False,False,False,False,False,None,None,None,None
4619,False,False,False,False,False,False,None,None,None,None,None
4620,False,False,False,False,None,None,None,None,None,None,None


In [34]:
df.head()

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,Apple,None,None,None,None,None,None,None,None,None
3,1,1,Chips and TomatilloGreen Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None


# 7. Order Id Integrity:
Question: Cross-reference the Order ID column for integrity. Are there any irregularities or patterns that need validation?


In [35]:
# Checkiing for unique Order Ids
unique_orders = df['Order_Id'].unique()


In [36]:
# Checking for irregularities
if len(unique_orders) == df.shape[0]:
    print("Order IDs are unique and consistent.")
else:
    print("There are irregularities or duplicate Order IDs.")

There are irregularities or duplicate Order IDs.


In [37]:
# Checking for missing values
missing_values = df['Order_Id'].isnull().sum()
if missing_values > 0:
    print(f"There are {missing_values} missing Order IDs.")
else:
    print("No missing Order IDs.")

No missing Order IDs.


In [38]:
# Checking for patterns or irregularities in Order IDs
order_id_counts = df['Order_Id'].value_counts()
print("Counts of each Order ID:")
print(order_id_counts)

Counts of each Order ID:
926     20
1483    14
1786    11
691     11
759     11
        ..
1334     1
1336     1
1339     1
59       1
1004     1
Name: Order_Id, Length: 1827, dtype: int64


In [39]:
#checking for order_Id integrity
df[df['Order_Id'] == 759]

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
1872,759,1,Chicken Bowl,11.25,Fresh Tomato Salsa,Rice,Black Beans,Pinto Beans,Guacamole,Lettuce,None,None,None,None
1873,759,1,Carnitas Bowl,9.25,Fresh Tomato Salsa,Rice,Pinto Beans,Cheese,Lettuce,None,None,None,None,None
1874,759,1,Chicken Soft Tacos,8.75,Fresh Tomato Salsa,Cheese,Lettuce,None,None,None,None,None,None,None
1875,759,1,Veggie Bowl,11.25,Roasted Chili Corn Salsa,Rice,Black Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None
1876,759,1,Chicken Bowl,8.75,Tomatillo Green Chili Salsa,Rice,Black Beans,Cheese,Sour Cream,Lettuce,None,None,None,None
1877,759,1,Chicken Bowl,11.25,Fresh Tomato Salsa,Fajita Vegetables,Rice,Guacamole,Lettuce,None,None,None,None,None
1878,759,2,Canned Soft Drink,2.50,Coke,None,None,None,None,None,None,None,None,None
1879,759,2,Canned Soft Drink,2.50,Diet Coke,None,None,None,None,None,None,None,None,None
1880,759,4,Bottled Water,6.00,None,None,None,None,None,None,None,None,None,None
1881,759,2,Chips and Guacamole,8.90,None,None,None,None,None,None,None,None,None,None


Justification: Each row represents a separate item within the same order, this might be accurate

# 8. Item Name Standardization:
Question: Standardize the Item Name column. Are there variations that can be unified for better analysis?


In [40]:
df['Item_Name']

0               Chips and Fresh Tomato Salsa
1                                       Izze
2                           Nantucket Nectar
3       Chips and TomatilloGreen Chili Salsa
4                               Chicken Bowl
                        ...                 
4617                           Steak Burrito
4618                           Steak Burrito
4619                      Chicken Salad Bowl
4620                      Chicken Salad Bowl
4621                      Chicken Salad Bowl
Name: Item_Name, Length: 4505, dtype: object

In [41]:
# Display unique values in the 'Item_Name' column
unique_item_names = df['Item_Name'].unique()
print(unique_item_names)


['Chips and Fresh Tomato Salsa' 'Izze' 'Nantucket Nectar'
 'Chips and TomatilloGreen Chili Salsa' 'Chicken Bowl' 'Side of Chips'
 'Steak Burrito' 'Steak Soft Tacos' 'Chips and Guacamole'
 'Chicken Crispy Tacos' 'Chicken Soft Tacos' 'Chicken Burrito'
 'Canned Soda' 'Barbacoa Burrito' 'Carnitas Burrito' 'Carnitas Bowl'
 'Bottled Water' 'Chips and Tomatillo Green Chili Salsa' 'Barbacoa Bowl'
 'Chips' 'Chicken Salad Bowl' 'Steak Bowl' 'Barbacoa Soft Tacos'
 'Veggie Burrito' 'Veggie Bowl' 'Steak Crispy Tacos'
 'Chips and Tomatillo Red Chili Salsa' 'Barbacoa Crispy Tacos'
 'Veggie Salad Bowl' 'Chips and Roasted ChiliCorn Salsa'
 'Chips and Roasted Chili Corn Salsa' 'Carnitas Soft Tacos'
 'Chicken Salad' 'Canned Soft Drink' 'Steak Salad Bowl'
 '6 Pack Soft Drink' 'Chips and TomatilloRed Chili Salsa' 'Bowl' 'Burrito'
 'Crispy Tacos' 'Carnitas Crispy Tacos' 'Steak Salad'
 'Chips and Mild Fresh Tomato Salsa' 'Veggie Soft Tacos'
 'Carnitas Salad Bowl' 'Barbacoa Salad Bowl' 'Salad' 'Veggie Crispy 

In [42]:
# Defining a regex pattern to match non-alphanumeric characters
pattern = r'[^a-zA-Z0-9\s]'

In [43]:
# Displaying cleaned 'Item_Name' column
df['Item_Name']

0               Chips and Fresh Tomato Salsa
1                                       Izze
2                           Nantucket Nectar
3       Chips and TomatilloGreen Chili Salsa
4                               Chicken Bowl
                        ...                 
4617                           Steak Burrito
4618                           Steak Burrito
4619                      Chicken Salad Bowl
4620                      Chicken Salad Bowl
4621                      Chicken Salad Bowl
Name: Item_Name, Length: 4505, dtype: object

In [44]:
# Removing special characters from 'Item_Name' column directly
df['Item_Name']=df['Item_Name'].str.replace(pattern, '', regex=True)
df['Item_Name']

0               Chips and Fresh Tomato Salsa
1                                       Izze
2                           Nantucket Nectar
3       Chips and TomatilloGreen Chili Salsa
4                               Chicken Bowl
                        ...                 
4617                           Steak Burrito
4618                           Steak Burrito
4619                      Chicken Salad Bowl
4620                      Chicken Salad Bowl
4621                      Chicken Salad Bowl
Name: Item_Name, Length: 4505, dtype: object

# 9. Quantity and Price Relationships & 13. Consistent Quantity and Price Units:

Question: Investigate the relationships between Quantity and Item Price. Are there cases where adjustments need to be made for accurate analysis?


In [45]:
df[['Quantity','Item_Price']]

,Quantity,Item_Price
0,1,2.39
1,1,3.39
2,1,3.39
3,1,2.39
4,2,16.98
...,...,...
4617,1,11.75
4618,1,11.75
4619,1,11.25
4620,1,8.75


In [46]:
# Summary statistics for Quantity and Item_Price columns
df[['Quantity', 'Item_Price']].describe()

,Quantity,Item_Price
count,4505.000000,4505.000000
mean,1.076138,7.514337
std,0.405580,4.243410
min,1.000000,1.090000
25%,1.000000,3.990000
50%,1.000000,8.750000
75%,1.000000,9.390000
max,15.000000,44.250000


In [47]:
# Finding unique items and their costs
item_costs = df.groupby('Item_Name')['Item_Price'].mean().to_dict()


In [48]:
# Calculating expected price based on Quantity and mean cost per item
expected_price = df['Quantity'] * df['Item_Name'].map(item_costs)

# Checking and replacing inconsistent prices
inconsistent_prices = abs(expected_price - df['Item_Price']) > 0.01  # Setting a threshold for inconsistency
df.loc[inconsistent_prices, 'Item_Price'] = expected_price[inconsistent_prices]


In [49]:
df['Item_price']=df['Item_Price'].round(2)

In [50]:
#Rounding off the decimals of Item Price Column
df['Item_Price']=df['Item_Price'].round(2)

In [51]:
df

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None,3.29
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None,3.39
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None,3.64
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None,2.54
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,10.46
4618,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,None,10.46
4619,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Pinto Beans,Guacamole,Lettuce,None,None,None,None,None,11.17
4620,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Lettuce,None,None,None,None,None,None,None,11.17


# 12. Handling Categorical Data:
Question: For categorical columns (e.g., Item Name), consider encoding or transforming them into a format suitable for analysis.


In [52]:
df['Item_Name'].head(20)

0             Chips and Fresh Tomato Salsa
1                                     Izze
2                         Nantucket Nectar
3     Chips and TomatilloGreen Chili Salsa
4                             Chicken Bowl
5                             Chicken Bowl
6                            Side of Chips
7                            Steak Burrito
8                         Steak Soft Tacos
9                            Steak Burrito
10                     Chips and Guacamole
11                    Chicken Crispy Tacos
12                      Chicken Soft Tacos
13                            Chicken Bowl
14                     Chips and Guacamole
15    Chips and TomatilloGreen Chili Salsa
16                         Chicken Burrito
17                         Chicken Burrito
18                             Canned Soda
19                            Chicken Bowl
Name: Item_Name, dtype: object

In [53]:
#Importing the LabelEncodder
from sklearn.preprocessing import LabelEncoder

In [54]:
# Initializing the LabelEncoder
label_encoder = LabelEncoder()

In [55]:
df.head()

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None,3.29
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None,3.39
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None,3.64
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None,2.54
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27


In [56]:
# Encoding the 'Item_Name' column
encoded_items = label_encoder.fit_transform(df['Item_Name'])

In [57]:
# Displaying the updated DataFrame with the encoded column
df['Encoded_Item'] = encoded_items
(df)

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price,Encoded_Item
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None,3.29,24
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None,3.39,34
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None,3.64,35
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None,2.54,31
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,10.46,39
4618,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,None,10.46,39
4619,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Pinto Beans,Guacamole,Lettuce,None,None,None,None,None,11.17,21
4620,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Lettuce,None,None,None,None,None,None,None,11.17,21


In [58]:
# Mapping encoded integers back to item names
decoded_mapping = {label: item for label, item in zip(encoded_items, df['Item_Name'])}

In [59]:
# Encoding the 'Item_Name' column
encoded_items = label_encoder.fit_transform(df['Item_Name'])

In [60]:
# Displaying the mapping of encoded integers back to item names
for label, item in decoded_mapping.items():
    print(f"Encoded: {label} -> Item: {item}")

Encoded: 24 -> Item: Chips and Fresh Tomato Salsa
Encoded: 34 -> Item: Izze
Encoded: 35 -> Item: Nantucket Nectar
Encoded: 31 -> Item: Chips and TomatilloGreen Chili Salsa
Encoded: 17 -> Item: Chicken Bowl
Encoded: 37 -> Item: Side of Chips
Encoded: 39 -> Item: Steak Burrito
Encoded: 43 -> Item: Steak Soft Tacos
Encoded: 25 -> Item: Chips and Guacamole
Encoded: 19 -> Item: Chicken Crispy Tacos
Encoded: 22 -> Item: Chicken Soft Tacos
Encoded: 18 -> Item: Chicken Burrito
Encoded: 9 -> Item: Canned Soda
Encoded: 2 -> Item: Barbacoa Burrito
Encoded: 12 -> Item: Carnitas Burrito
Encoded: 11 -> Item: Carnitas Bowl
Encoded: 6 -> Item: Bottled Water
Encoded: 29 -> Item: Chips and Tomatillo Green Chili Salsa
Encoded: 1 -> Item: Barbacoa Bowl
Encoded: 23 -> Item: Chips
Encoded: 21 -> Item: Chicken Salad Bowl
Encoded: 38 -> Item: Steak Bowl
Encoded: 5 -> Item: Barbacoa Soft Tacos
Encoded: 45 -> Item: Veggie Burrito
Encoded: 44 -> Item: Veggie Bowl
Encoded: 40 -> Item: Steak Crispy Tacos
Encoded: 

In [61]:
df.head(30)

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price,Encoded_Item
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None,3.29,24
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None,3.39,34
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None,3.64,35
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None,2.54,31
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27,17
5,3,1,Chicken Bowl,10.13,Fresh Tomato Salsa Mild,Rice,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None,None,10.13,17
6,3,1,Side of Chips,1.84,None,None,None,None,None,None,None,None,None,None,1.84,37
7,4,1,Steak Burrito,10.46,Tomatillo Red Chili Salsa,Fajita Vegetables,Black Beans,Pinto Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None,10.46,39
8,4,1,Steak Soft Tacos,9.75,Tomatillo Green Chili Salsa,Pinto Beans,Cheese,Sour Cream,Lettuce,None,None,None,None,None,9.75,43
9,5,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Black Beans,Pinto Beans,Cheese,Sour Cream,Lettuce,None,None,None,10.46,39


# 11. Converting to CSV:
Optional Question: If needed, convert the cleaned dataset to a CSV file using the to_csv function with sep=','.


In [62]:
# Saving the cleaned dataset to a CSV file
df.to_csv('chipotle_cleaned_dataset.csv', sep=',', index=False)

In [63]:
cleant_data=pd.read_csv("chipotle_cleaned_dataset.csv")
cleant_data

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price,Encoded_Item
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.29,24
1,1,1,Izze,3.39,Clementine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.39,34
2,1,1,Nantucket Nectar,3.64,Apple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.64,35
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.54,31
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,NaN,NaN,NaN,NaN,NaN,20.27,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4500,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole,NaN,NaN,NaN,10.46,39
4501,1833,1,Steak Burrito,10.46,Fresh Tomato Salsa,Rice,Sour Cream,Cheese,Lettuce,Guacamole,NaN,NaN,NaN,NaN,10.46,39
4502,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Pinto Beans,Guacamole,Lettuce,NaN,NaN,NaN,NaN,NaN,11.17,21
4503,1834,1,Chicken Salad Bowl,11.17,Fresh Tomato Salsa,Fajita Vegetables,Lettuce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.17,21


# Quantity of each item? (Top 5 itmes available in store)

In [64]:
df.head()

,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price,Encoded_Item
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None,3.29,24
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None,3.39,34
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None,3.64,35
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None,2.54,31
4,2,2,Chicken Bowl,20.27,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27,17


In [65]:
# Grouping by 'Item_Name', summing quantities, and calculating total price
item_counts = df.groupby('Item_Name').agg({
    'Quantity': 'sum',
    'Item_Price': 'sum'
}).reset_index()

In [66]:
# Sorting by quantity and price to get top 5 items
top_5_by_quantity = item_counts.sort_values(by='Quantity', ascending=False).head(5)
top_5_by_price = item_counts.sort_values(by='Item_Price', ascending=False).head(5)


In [67]:
print("Top 5 items by Quantity:")
print(top_5_by_quantity)

Top 5 items by Quantity:
              Item_Name  Quantity  Item_Price
17         Chicken Bowl       744     7537.05
18      Chicken Burrito       577     5827.70
25  Chips and Guacamole       496     2281.58
39        Steak Burrito       380     3974.80
10    Canned Soft Drink       331      486.67


In [68]:
print("\nTop 5 items by Total Price:")
print(top_5_by_price)


Top 5 items by Total Price:
              Item_Name  Quantity  Item_Price
17         Chicken Bowl       744     7537.05
18      Chicken Burrito       577     5827.70
39        Steak Burrito       380     3974.80
38           Steak Bowl       219     2349.78
25  Chips and Guacamole       496     2281.58


# which is most ordered item in choice_description?

In [69]:
# Concatenating all 'Choice_Description' columns
choice_columns = [col for col in df.columns if 'Choice_Description_' in col]
all_choices = df[choice_columns].values.flatten()

# Creating a Series with all choices and counting occurrences
choices_series = pd.Series(all_choices)
most_ordered_item = choices_series.value_counts().idxmax()

print("The most ordered item in 'Choice_Description' is:", most_ordered_item)

The most ordered item in 'Choice_Description' is:  Rice


 # Calculating total order count


In [70]:
# Calculating total order count
total_orders = df['Order_Id'].nunique()

print("Total Order Count:", total_orders)

Total Order Count: 1827


# What is revenue?

In [71]:
# Calculating revenue
df['Total_Price'] = df['Quantity'] * df['Item_Price']
total_revenue = df['Total_Price'].sum()

print("Total Revenue:", total_revenue)

Total Revenue: 40975.32


# What is average revenue amount per order?

In [72]:
#Calculating number of orders
num_orders = df['Order_Id'].nunique()

In [73]:
# Calculating average revenue per order
average_revenue_per_order = total_revenue / num_orders

In [74]:
print("Average Revenue Amount per Order:", average_revenue_per_order)

Average Revenue Amount per Order: 22.427651888341543


# How many different items are sold?

In [75]:
# Calculating number of different items sold
num_different_items = df['Item_Name'].nunique()

print("Number of Different Items Sold:", num_different_items)

Number of Different Items Sold: 50


# what is revenue from each product?

In [76]:
# Calculating total revenue for each product
df['Total_Price'] = df['Quantity'] * df['Item_Price']
revenue_per_product = df.groupby('Item_Name')['Total_Price'].sum()

print("Revenue from Each Product:")
print(revenue_per_product)

Revenue from Each Product:
Item_Name
6 Pack Soft Drink                         376.77
Barbacoa Bowl                             654.08
Barbacoa Burrito                          876.65
Barbacoa Crispy Tacos                     153.02
Barbacoa Salad Bowl                        87.60
Barbacoa Soft Tacos                       250.50
Bottled Water                             698.14
Bowl                                      148.00
Burrito                                    44.40
Canned Soda                               228.76
Canned Soft Drink                         680.94
Carnitas Bowl                             855.62
Carnitas Burrito                          628.08
Carnitas Crispy Tacos                     111.38
Carnitas Salad                              8.99
Carnitas Salad Bowl                        66.36
Carnitas Soft Tacos                       376.00
Chicken Bowl                             8287.02
Chicken Burrito                          6756.90
Chicken Crispy Tacos            

# Find the unique items

In [77]:
# Finding unique items
unique_items = df['Item_Name'].unique()

print("Unique Items:")
print(unique_items)

Unique Items:
['Chips and Fresh Tomato Salsa' 'Izze' 'Nantucket Nectar'
 'Chips and TomatilloGreen Chili Salsa' 'Chicken Bowl' 'Side of Chips'
 'Steak Burrito' 'Steak Soft Tacos' 'Chips and Guacamole'
 'Chicken Crispy Tacos' 'Chicken Soft Tacos' 'Chicken Burrito'
 'Canned Soda' 'Barbacoa Burrito' 'Carnitas Burrito' 'Carnitas Bowl'
 'Bottled Water' 'Chips and Tomatillo Green Chili Salsa' 'Barbacoa Bowl'
 'Chips' 'Chicken Salad Bowl' 'Steak Bowl' 'Barbacoa Soft Tacos'
 'Veggie Burrito' 'Veggie Bowl' 'Steak Crispy Tacos'
 'Chips and Tomatillo Red Chili Salsa' 'Barbacoa Crispy Tacos'
 'Veggie Salad Bowl' 'Chips and Roasted ChiliCorn Salsa'
 'Chips and Roasted Chili Corn Salsa' 'Carnitas Soft Tacos'
 'Chicken Salad' 'Canned Soft Drink' 'Steak Salad Bowl'
 '6 Pack Soft Drink' 'Chips and TomatilloRed Chili Salsa' 'Bowl' 'Burrito'
 'Crispy Tacos' 'Carnitas Crispy Tacos' 'Steak Salad'
 'Chips and Mild Fresh Tomato Salsa' 'Veggie Soft Tacos'
 'Carnitas Salad Bowl' 'Barbacoa Salad Bowl' 'Salad' '

# Find total cost and quantity of every unique items

In [78]:
# Calculating total cost and quantity for each unique item
df['Total_Cost'] = df['Quantity'] * df['Item_Price']
item_summary = df.groupby('Item_Name').agg({
    'Quantity': 'sum',
    'Total_Cost': 'sum'
}).reset_index()

print("Total Cost and Quantity of Every Unique Item:")
print(item_summary)

Total Cost and Quantity of Every Unique Item:
                                Item_Name  Quantity  Total_Cost
0                       6 Pack Soft Drink        55      376.77
1                           Barbacoa Bowl        64      654.08
2                        Barbacoa Burrito        89      876.65
3                   Barbacoa Crispy Tacos        12      153.02
4                     Barbacoa Salad Bowl         8       87.60
5                     Barbacoa Soft Tacos        25      250.50
6                           Bottled Water       188      698.14
7                                    Bowl         4      148.00
8                                 Burrito         6       44.40
9                             Canned Soda       122      228.76
10                      Canned Soft Drink       331      680.94
11                          Carnitas Bowl        71      855.62
12                       Carnitas Burrito        60      628.08
13                  Carnitas Crispy Tacos         8      1

# Find the average of every unique item

In [79]:
# Calculating average quantity and average cost per unit for each unique item
item_summary = df.groupby('Item_Name').agg({
    'Quantity': 'mean',
    'Item_Price': 'mean'
}).reset_index()

print("Average Quantity and Average Cost per Unit of Every Unique Item:")
print('\n',item_summary)

Average Quantity and Average Cost per Unit of Every Unique Item:

                                 Item_Name  Item_Price
0                       6 Pack Soft Drink    6.732407
1                           Barbacoa Bowl   10.220000
2                        Barbacoa Burrito    9.850000
3                   Barbacoa Crispy Tacos   11.923636
4                     Barbacoa Salad Bowl   10.950000
5                     Barbacoa Soft Tacos   10.020000
6                           Bottled Water    2.367466
7                                    Bowl   29.600000
8                                 Burrito    7.400000
9                             Canned Soda    1.622600
10                      Canned Soft Drink    1.731922
11                          Carnitas Bowl   11.308088
12                       Carnitas Burrito   10.301864
13                  Carnitas Crispy Tacos   12.730000
14                         Carnitas Salad    8.990000
15                    Carnitas Salad Bowl   11.060000
16             

# Replace the items if abnormal with avg

In [80]:
# Define thresholds for abnormal values
quantity_threshold = 5  # Example threshold for abnormal quantity
price_threshold = 5.0  # Example threshold for abnormal price

# Calculating average quantity and average price
avg_quantity = df['Quantity'].mean()
avg_price = df['Item_Price'].mean()

# Replacing abnormal quantities and prices with averages
df.loc[df['Quantity'] > quantity_threshold, 'Quantity'] = avg_quantity
df.loc[df['Item_Price'] > price_threshold, 'Item_Price'] = avg_price

print("DataFrame after replacing abnormal values:")
df

DataFrame after replacing abnormal values:


,Order_Id,Quantity,Item_Name,Item_Price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10,Item_price,Encoded_Item,Total_Price,Total_Cost
0,1,1.0,Chips and Fresh Tomato Salsa,3.290000,None,None,None,None,None,None,None,None,None,None,3.29,24,3.29,3.29
1,1,1.0,Izze,3.390000,Clementine,None,None,None,None,None,None,None,None,None,3.39,34,3.39,3.39
2,1,1.0,Nantucket Nectar,3.640000,Apple,None,None,None,None,None,None,None,None,None,3.64,35,3.64,3.64
3,1,1.0,Chips and TomatilloGreen Chili Salsa,2.540000,None,None,None,None,None,None,None,None,None,None,2.54,31,2.54,2.54
4,2,2.0,Chicken Bowl,7.942111,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None,20.27,17,40.54,40.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,1833,1.0,Steak Burrito,7.942111,Fresh Tomato Salsa,Rice,Black Beans,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,10.46,39,10.46,10.46
4618,1833,1.0,Steak Burrito,7.942111,Fresh Tomato Salsa,Rice,Sour Cream,Cheese,Lettuce,Guacamole,None,None,None,None,10.46,39,10.46,10.46
4619,1834,1.0,Chicken Salad Bowl,7.942111,Fresh Tomato Salsa,Fajita Vegetables,Pinto Beans,Guacamole,Lettuce,None,None,None,None,None,11.17,21,11.17,11.17
4620,1834,1.0,Chicken Salad Bowl,7.942111,Fresh Tomato Salsa,Fajita Vegetables,Lettuce,None,None,None,None,None,None,None,11.17,21,11.17,11.17
